# NDMI Kepler

In [1]:
import io
import os
import sys

import geopandas as gpd
import pandas as pd
from keplergl import KeplerGl

module_path = os.path.abspath(os.path.join("../../"))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/Repos/observatorio-ciudades/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Downlaod data

NDMI

In [2]:
schema = 'raster_analysis'
table = 'ndmi_analysis_hex'

city = 'CDMX'

query = f"SELECT * FROM {schema}.{table} WHERE \"city\" = '{city}' AND \"res\" = {10}"

hex_gdf = aup.gdf_from_query(query) 
print(hex_gdf.shape)
hex_gdf.head(2)

(64174, 11)


,hex_id,geometry,res,ndmi_mean,ndmi_std,ndmi_median,ndmi_max,ndmi_min,ndmi_diff,ndmi_tend,city
0,8a4995959c17fff,"POLYGON ((-99.16973 19.18997, -99.16908 19.190...",10,0.059833,0.052246,0.063521,0.133204,-0.01477,0.147973,0.000434,CDMX
1,8a499595bb47fff,"POLYGON ((-99.16604 19.20465, -99.16539 19.205...",10,0.032387,0.115082,0.005131,0.199716,-0.09263,0.292346,0.000628,CDMX


Urban data

In [3]:
schema = 'hexgrid'
table = 'hexgrid_10_city_2020'

city = 'CDMX'
tipo_entorno = 'urban'

query = f"SELECT * FROM {schema}.{table} WHERE \"city\" = '{city}' AND \"type\" = '{tipo_entorno}'"

hex_urb = aup.gdf_from_query(query) 
print(hex_urb.shape)
hex_urb.head(2)

(47502, 6)


,hex_id_10,geometry,CVEGEO,NOMGEO,city,type
0,8a4995b8cb57fff,"POLYGON ((-99.19358 19.48193, -99.19293 19.482...",09002,Azcapotzalco,CDMX,urban
1,8a4995bab38ffff,"POLYGON ((-99.20837 19.48081, -99.20772 19.481...",09002,Azcapotzalco,CDMX,urban


## Filter for urban area

In [4]:
hex_idx = list(hex_urb.hex_id_10.unique())
hex_gdf = hex_gdf.loc[hex_gdf.hex_id.isin(hex_idx)].copy()
print(hex_gdf.shape)

(47502, 11)


## Create Kepler

Data treatment

<0.1	Bare soil
0.1 – 0.2	Almost absent canopy cover
0.2 – 0.3	Very low canopy cover
0.3 – 0.4	Low canopy cover, low vigour or very low canopy cover, high vigour
0.4 – 0.5	Mid-low canopy cover, low vigour or low canopy cover, high vigour
0.5 – 0.6	Average canopy cover, low vigour or mid-low canopy cover, high vigour
0.6 – 0.7	Mid-high canopy cover, low vigour or average canopy cover, high vigour
0.7 – 0.8	High canopy cover, high vigour
0.8 – 0.9	Very high canopy cover, very high vigour
0.9 – 1.0	Total canopy cover, very high vigour

In [5]:
hex_gdf['ndmi_mean'] = hex_gdf['ndmi_mean'].round(decimals = 3)
hex_gdf = hex_gdf.rename(columns={'ndmi_mean':'Índice de humedad'})

In [6]:
hex_gdf['ndmi_diff'] = hex_gdf['ndmi_diff'].round(decimals = 3)
hex_gdf = hex_gdf.rename(columns={'ndmi_diff':'Variación de humedad anual'})

In [7]:
hex_gdf.head(2)

,hex_id,geometry,res,Índice de humedad,ndmi_std,ndmi_median,ndmi_max,ndmi_min,Variación de humedad anual,ndmi_tend,city
4,8a4995959437fff,"POLYGON ((-99.16617 19.19689, -99.16552 19.197...",10,0.074,0.085720,0.048988,0.239059,-0.027803,0.267,0.000282,CDMX
8,8a499595b90ffff,"POLYGON ((-99.15871 19.19635, -99.15807 19.196...",10,0.050,0.094892,0.042274,0.195947,-0.076172,0.272,0.000439,CDMX


### Configure data

In [14]:
config,config_idx = aup.kepler_config()

# column = 'Variación de humedad anual'
column = 'Índice de humedad'

config["config"]["visState"]["layers"][0]["visualChannels"]["colorField"]["name"] = column
config["config"]["visState"]["interactionConfig"]["tooltip"][
"fieldsToShow"]["Análisis de hexágono"] = [column]
layer_id = 0
config[list(config.keys())[1]][list(config[list(config.keys())[1]].keys())[0]]['layers'][layer_id]['config']['visConfig']['opacity'] = 0.35

# for Variacion anual
if column == 'Variación de humedad anual':
    config[list(config.keys())[1]][list(config[list(config.keys())[1]].keys())[0]]['layers'][layer_id]['config']['visConfig']['colorRange']['colors'] = ['#00939c','#85c4c8','#feeee8','#ec9370','#c22e00']

### Create Kepler

In [15]:
# hex_gdf["d"] = 0
hex_kepler = hex_gdf.copy()
hex_kepler = hex_kepler[[column,'geometry']]
longitude = hex_gdf.dissolve().geometry.centroid.x
latitude = hex_gdf.dissolve().geometry.centroid.y
config["config"]["mapState"]["latitude"] = latitude[0]
config["config"]["mapState"]["longitude"] = longitude[0]
config["config"]["mapState"]["zoom"] = 10.25

# hex_kepler['           '] = '         '
# hex_kepler['            '] = '          '
# session = boto3.Session(profile_name="observatorio")
# dev_s3_client = session.client("s3")
# map_city = KeplerGl(height=800, data={"Análisis de ciudad": hex_kepler}, config=config_idx)
map_city = KeplerGl(height=500)
map_city.config = config
map_city.add_data(hex_kepler, name='Análisis de hexágono')
map_city.save_to_html(file_name="cdmx_ndmi.html", read_only=False)

/tmp/ipykernel_41858/1842880563.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_gdf.dissolve().geometry.centroid.x
/tmp/ipykernel_41858/1842880563.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_gdf.dissolve().geometry.centroid.y


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to cdmx_ndmi.html!
